# SAMOS Spectroscopy - Visual QA & Inspection

**Interactive quality assessment and parameter tuning**

## Purpose

This notebook provides:
1. Visual inspection of all extracted slits
2. Quality assessment of trace detection
3. Reference slit selection for wavelength calibration
4. Bad slit flagging
5. QA report generation

## Prerequisites

- Run `01_initial_inspection.ipynb` first
- Creates `spec_*.fits` files in working directory

---

## 1. Import Packages

In [ ]:
# Standard libraries
import os
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path
from astropy.io import fits
from astropy.stats import sigma_clipped_stats

# SAMOS package
from samos.utils import display, io

# Jupyter display
%matplotlib inline
plt.rcParams['figure.figsize'] = (16, 8)

print("✓ Packages loaded")

## 2. Configuration

In [ ]:
# Target information (should match notebook 01)
target_name = "ABELL3120"
target_mode = "SAMI_manual_Mask_T00_Low_Red"

# Working directory
analysis_top_directory = "/Users/nestrada/Documents/SAMOS/SAMOS_REDUCED"
working_directory = os.path.join(analysis_top_directory, target_name, target_mode)

os.chdir(working_directory)
print(f"Working directory: {working_directory}")

# Find all spec files
spec_files = sorted(Path('.').glob('spec_*.fits'))
n_slits = len(spec_files)

print(f"\nFound {n_slits} slit files")
print(f"Files: {spec_files[0]} to {spec_files[-1]}")

## 3. Quick Overview - All Slits

Display all slits in a grid to get an overview.

In [ ]:
# Determine grid layout
n_cols = 3
n_rows = (n_slits + n_cols - 1) // n_cols

fig, axes = plt.subplots(n_rows, n_cols, figsize=(20, 4*n_rows))
axes = axes.flatten() if n_slits > 1 else [axes]

print("Loading and displaying all slits...\n")

for i, spec_file in enumerate(spec_files):
    # Load data
    with fits.open(spec_file) as hdul:
        data = hdul['DATA'].data
        header = hdul[0].header
        slit_num = header['SLIT']
    
    # Display
    vmin, vmax = display.get_percentile_limits(data, 1, 99)
    
    ax = axes[i]
    im = ax.imshow(data, origin='lower', cmap='gray', 
                   vmin=vmin, vmax=vmax, aspect='auto')
    ax.set_title(f'Slit {slit_num}', fontsize=10)
    ax.set_xlabel('X (dispersion)', fontsize=8)
    ax.set_ylabel('Y (spatial)', fontsize=8)
    
    if (i+1) % n_cols == 0 or i == n_slits - 1:
        print(f"  Displayed {min(i+1, n_slits)}/{n_slits} slits")

# Hide unused subplots
for j in range(n_slits, len(axes)):
    axes[j].axis('off')

plt.tight_layout()
plt.show()

print(f"\n✓ Overview complete")

## 4. Quality Assessment Metrics

Calculate quality metrics for each slit.

In [ ]:
import pandas as pd

# Calculate metrics for each slit
qa_data = []

for spec_file in spec_files:
    with fits.open(spec_file) as hdul:
        slit_num = hdul[0].header['SLIT']
        data = hdul['DATA'].data
        flat = hdul['FLAT'].data
        lines = hdul['LINES'].data
        
        # Calculate statistics
        mean_sci, median_sci, std_sci = sigma_clipped_stats(data, sigma=3.0)
        mean_flat, median_flat, std_flat = sigma_clipped_stats(flat, sigma=3.0)
        
        # Arc line strength (peak value)
        arc_peak = np.max(lines)
        
        # Estimate S/N (simple: median / std)
        snr = median_sci / std_sci if std_sci > 0 else 0
        
        qa_data.append({
            'Slit': slit_num,
            'Median_Sci': median_sci,
            'STD_Sci': std_sci,
            'S/N': snr,
            'Arc_Peak': arc_peak,
            'Flat_Median': median_flat,
            'Shape': f"{data.shape[0]}x{data.shape[1]}"
        })

# Create DataFrame
qa_df = pd.DataFrame(qa_data)

print("Quality Assessment Metrics:")
print("=" * 80)
print(qa_df.to_string(index=False))
print("=" * 80)

# Summary statistics
print(f"\nSummary:")
print(f"  Average S/N: {qa_df['S/N'].mean():.1f}")
print(f"  S/N range: {qa_df['S/N'].min():.1f} - {qa_df['S/N'].max():.1f}")
print(f"  Average arc peak: {qa_df['Arc_Peak'].mean():.0f}")

## 5. Select Reference Slit for Wavelength Calibration

Choose the best slit for wavelength calibration based on arc lamp brightness.

In [ ]:
# Find slit with brightest arc lines
best_slit_idx = qa_df['Arc_Peak'].idxmax()
reference_slit = qa_df.loc[best_slit_idx, 'Slit']
reference_arc_peak = qa_df.loc[best_slit_idx, 'Arc_Peak']

print(f"Recommended reference slit: {reference_slit}")
print(f"  Arc peak intensity: {reference_arc_peak:.0f}")
print(f"  This slit will be used for wavelength calibration in notebook 03")

# Plot arc spectrum quality
fig, ax = plt.subplots(figsize=(12, 5))
ax.bar(qa_df['Slit'], qa_df['Arc_Peak'], color='steelblue', alpha=0.7)
ax.axhline(qa_df['Arc_Peak'].mean(), color='red', linestyle='--', 
          label=f'Mean = {qa_df["Arc_Peak"].mean():.0f}')
ax.scatter([reference_slit], [reference_arc_peak], color='red', s=200, 
          marker='*', label=f'Reference slit {reference_slit}', zorder=10)
ax.set_xlabel('Slit Number')
ax.set_ylabel('Arc Line Peak Intensity')
ax.set_title('Arc Lamp Quality by Slit')
ax.legend()
ax.grid(alpha=0.3)
plt.show()

print(f"\n✓ Reference slit selected: {reference_slit}")

## 6. Detailed View - Individual Slits

Examine individual slits in detail.

In [ ]:
def display_slit_detail(slit_number):
    """Display detailed view of a single slit."""
    spec_file = f'spec_{slit_number:03d}.fits'
    
    if not Path(spec_file).exists():
        print(f"⚠️ File not found: {spec_file}")
        return
    
    with fits.open(spec_file) as hdul:
        data = hdul['DATA'].data
        flat = hdul['FLAT'].data
        lines = hdul['LINES'].data
        header = hdul[0].header
    
    # Create figure with subplots
    fig = plt.figure(figsize=(20, 12))
    
    # Science data
    ax1 = plt.subplot(3, 2, 1)
    vmin, vmax = display.get_percentile_limits(data, 1, 99)
    im1 = ax1.imshow(data, origin='lower', cmap='gray', 
                     vmin=vmin, vmax=vmax, aspect='auto')
    ax1.set_title(f'Slit {slit_number} - Science Data')
    ax1.set_xlabel('X (dispersion)')
    ax1.set_ylabel('Y (spatial)')
    plt.colorbar(im1, ax=ax1, label='Counts')
    
    # Flat field
    ax2 = plt.subplot(3, 2, 2)
    vmin, vmax = display.get_percentile_limits(flat, 1, 99)
    im2 = ax2.imshow(flat, origin='lower', cmap='gray', 
                     vmin=vmin, vmax=vmax, aspect='auto')
    ax2.set_title(f'Slit {slit_number} - Flat Field')
    ax2.set_xlabel('X (dispersion)')
    ax2.set_ylabel('Y (spatial)')
    plt.colorbar(im2, ax=ax2, label='Counts')
    
    # Arc lamp
    ax3 = plt.subplot(3, 2, 3)
    vmin, vmax = display.get_percentile_limits(lines, 1, 99)
    im3 = ax3.imshow(lines, origin='lower', cmap='gray', 
                     vmin=vmin, vmax=vmax, aspect='auto')
    ax3.set_title(f'Slit {slit_number} - Arc Lamp')
    ax3.set_xlabel('X (dispersion)')
    ax3.set_ylabel('Y (spatial)')
    plt.colorbar(im3, ax=ax3, label='Counts')
    
    # Spatial profiles
    ax4 = plt.subplot(3, 2, 4)
    x_mid = data.shape[1] // 2
    ax4.plot(data[:, x_mid], label='Science', alpha=0.7)
    ax4.plot(flat[:, x_mid], label='Flat', alpha=0.7)
    ax4.plot(lines[:, x_mid], label='Arc', alpha=0.7)
    ax4.set_xlabel('Y pixel')
    ax4.set_ylabel('Counts')
    ax4.set_title(f'Spatial Profiles (x={x_mid})')
    ax4.legend()
    ax4.grid(alpha=0.3)
    
    # Spectral profile (collapsed)
    ax5 = plt.subplot(3, 2, 5)
    spectrum = np.sum(data, axis=0)
    ax5.plot(spectrum, label='Science')
    ax5.set_xlabel('X pixel (dispersion)')
    ax5.set_ylabel('Counts')
    ax5.set_title('Collapsed Spectrum')
    ax5.grid(alpha=0.3)
    
    # Arc line profile (collapsed)
    ax6 = plt.subplot(3, 2, 6)
    arc_spectrum = np.sum(lines, axis=0)
    ax6.plot(arc_spectrum, label='Arc', color='orange')
    ax6.set_xlabel('X pixel (dispersion)')
    ax6.set_ylabel('Counts')
    ax6.set_title('Arc Line Spectrum')
    ax6.grid(alpha=0.3)
    
    plt.tight_layout()
    plt.show()
    
    # Print metadata
    print(f"\nSlit {slit_number} Metadata:")
    print(f"  Shape: {data.shape}")
    print(f"  Y boundaries: [{header['Y0']}, {header['Y1']}, {header['Y2']}, {header['Y3']}]")
    print(f"  Science median: {np.median(data):.1f}")
    print(f"  Arc peak: {np.max(lines):.0f}")

# Display reference slit
print(f"Displaying reference slit {reference_slit}:\n")
display_slit_detail(reference_slit)

### Inspect Other Slits

Change the slit number below to inspect other slits.

In [ ]:
# Change this to inspect different slits
slit_to_inspect = 0

display_slit_detail(slit_to_inspect)

## 7. Flag Bad Slits (Optional)

Manually flag slits that should be excluded from processing.

In [ ]:
# List of bad slits to exclude (edit as needed)
bad_slits = []

# Reasons for flagging (optional documentation)
bad_slit_reasons = {
    # Example: 5: "Low S/N",
    # Example: 12: "Cosmic ray contamination",
}

if len(bad_slits) > 0:
    print(f"Flagged {len(bad_slits)} bad slit(s):")
    for slit in bad_slits:
        reason = bad_slit_reasons.get(slit, "No reason given")
        print(f"  Slit {slit}: {reason}")
else:
    print("No bad slits flagged (all slits will be processed)")

# Good slits for processing
good_slits = [s for s in qa_df['Slit'] if s not in bad_slits]
print(f"\nGood slits for processing: {len(good_slits)}/{n_slits}")

## 8. Save QA Configuration

Save configuration for notebook 03.

In [ ]:
import yaml

qa_config = {
    'target': target_name,
    'mode': target_mode,
    'n_slits': n_slits,
    'reference_slit': int(reference_slit),
    'bad_slits': [int(s) for s in bad_slits],
    'good_slits': [int(s) for s in good_slits],
    'qa_metrics': qa_df.to_dict('records')
}

# Save configuration
config_file = 'qa_config.yaml'
with open(config_file, 'w') as f:
    yaml.dump(qa_config, f, default_flow_style=False)

print(f"✓ QA configuration saved to: {config_file}")
print(f"\nConfiguration:")
print(f"  Reference slit: {reference_slit}")
print(f"  Good slits: {len(good_slits)}")
print(f"  Bad slits: {len(bad_slits)}")

## 9. Generate QA Report (Optional)

Create a PDF summary of the quality assessment.

In [ ]:
from matplotlib.backends.backend_pdf import PdfPages

# Create PDF report
pdf_file = 'qa_report.pdf'

with PdfPages(pdf_file) as pdf:
    # Page 1: Overview grid (recreate without showing)
    fig, axes = plt.subplots(n_rows, n_cols, figsize=(20, 4*n_rows))
    axes = axes.flatten() if n_slits > 1 else [axes]
    
    for i, spec_file in enumerate(spec_files):
        with fits.open(spec_file) as hdul:
            data = hdul['DATA'].data
            slit_num = hdul[0].header['SLIT']
        
        vmin, vmax = display.get_percentile_limits(data, 1, 99)
        ax = axes[i]
        ax.imshow(data, origin='lower', cmap='gray', 
                 vmin=vmin, vmax=vmax, aspect='auto')
        ax.set_title(f'Slit {slit_num}', fontsize=10)
        
        # Highlight bad slits
        if slit_num in bad_slits:
            ax.set_facecolor('lightcoral')
            ax.text(0.5, 0.5, 'BAD', transform=ax.transAxes,
                   fontsize=20, color='red', ha='center', va='center',
                   bbox=dict(boxstyle='round', facecolor='white', alpha=0.8))
        
        # Highlight reference slit
        if slit_num == reference_slit:
            for spine in ax.spines.values():
                spine.set_edgecolor('green')
                spine.set_linewidth(3)
    
    for j in range(n_slits, len(axes)):
        axes[j].axis('off')
    
    fig.suptitle(f'SAMOS Spectroscopy QA - {target_name}\n{target_mode}',
                fontsize=16, fontweight='bold')
    plt.tight_layout()
    pdf.savefig(fig)
    plt.close()
    
    # Page 2: QA metrics
    fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(12, 10))
    
    # S/N plot
    colors = ['red' if s in bad_slits else 'steelblue' for s in qa_df['Slit']]
    ax1.bar(qa_df['Slit'], qa_df['S/N'], color=colors, alpha=0.7)
    ax1.axhline(qa_df['S/N'].mean(), color='black', linestyle='--', 
               label=f'Mean S/N = {qa_df["S/N"].mean():.1f}')
    ax1.set_xlabel('Slit Number')
    ax1.set_ylabel('S/N')
    ax1.set_title('Signal-to-Noise Ratio by Slit')
    ax1.legend()
    ax1.grid(alpha=0.3)
    
    # Arc quality plot
    colors = ['red' if s in bad_slits else 'steelblue' for s in qa_df['Slit']]
    ax2.bar(qa_df['Slit'], qa_df['Arc_Peak'], color=colors, alpha=0.7)
    ax2.scatter([reference_slit], [qa_df[qa_df['Slit']==reference_slit]['Arc_Peak'].values[0]], 
               color='green', s=200, marker='*', label=f'Reference slit {reference_slit}', zorder=10)
    ax2.set_xlabel('Slit Number')
    ax2.set_ylabel('Arc Peak Intensity')
    ax2.set_title('Arc Lamp Quality by Slit')
    ax2.legend()
    ax2.grid(alpha=0.3)
    
    plt.tight_layout()
    pdf.savefig(fig)
    plt.close()

print(f"✓ QA report saved to: {pdf_file}")
print(f"\nReport contains:")
print(f"  Page 1: Overview of all slits")
print(f"  Page 2: QA metrics plots")

## Summary

This notebook has:

1. ✓ Loaded and visualized all extracted slits
2. ✓ Calculated quality metrics (S/N, arc brightness)
3. ✓ Selected reference slit for wavelength calibration
4. ✓ Provided detailed inspection tools
5. ✓ Flagged bad slits (if any)
6. ✓ Saved QA configuration for notebook 03
7. ✓ Generated QA report (optional)

**Next Steps:**

Proceed to `03_spectroscopy_pypeit.ipynb` for:
- Trace fitting and rectification
- 1D spectrum extraction
- Wavelength calibration
- Batch processing of all good slits

**Files Created:**
- `qa_config.yaml` - Configuration for notebook 03
- `qa_report.pdf` - QA summary (if generated)